# How fast can your code be?

In this brief notebook, we will have a taste of how to speed up your code. Note, this is intended to be only an access point to the huge amount of resources (and dedicated lectures) on this extremely important topic. Go to the Acknowledgement section for further reading.

Let's start from a simple code that computes the (approximated) value of pi, and times the calculation.

In [1]:
import time
  
def Pi(num_steps):
   start = time.time()
   step = 1.0/num_steps
   sum = 0

   for i in range(num_steps):
      x= (i+0.5)*step
      sum = sum + 4.0/(1.0+x*x)


   pi = step * sum
   end = time.time()
   print("Pi with %d steps is %f in %f secs" %(num_steps, pi, end-start))

In [2]:
Pi(1000)

Pi with 1000 steps is 3.141593 in 0.000153 secs


Now try the same with C. Open your favourite editor, write the following code and save the doc as pi.c:

```
#include <stdio.h>
#include <time.h>

void Pi(int num_steps){
   double start, end, pi, step, x, sum;
   int i;
   start = clock();
   step = 1.0/(double)num_steps;
   sum = 0;
   for (i=0;i<num_steps;i++){
      x = (i+0.5)*step;
      sum = sum + 4.0/(1.0+x*x);
   }
   pi = step * sum;
   end= clock();
   printf("Pi with %d steps is %f in %f secs\n",num_steps, pi,(float)(end- start)/CLOCKS_PER_SEC);
}

int main() {
   Pi(1000);
   return 0;
}
```

Now compile and create the executable file. Run the following command in the terminal shell:
```
gcc -o pi -O3 pi.c
```

Then, execute:
```
./pi
```

What is the result? Is the C-implemented code faster or slower than the python code? Can you identify the bottleneck of this code?

# Optimization

Before 'going parallel', you can improve the performance of your code. Once you identify the bottleneck (see exercise above), you can check whether you can do to eliminate, or at least ease it. 

Let's separate the loop part from the rest of the function.

In [3]:
def loop(num_steps):
    step = 1.0/num_steps
    sum = 0
    
    for i in range(num_steps):
        x= (i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)
    return sum

def Pi(num_steps):
    start = time.time()
    sum = loop(num_steps)
    pi = sum/num_steps
    end = time.time()
    print("Pi with %d steps is %f in %f secs" %(num_steps, pi, end-start))

This is going to take the same time as above. Not surprising: we haven't done anything, just split the original function in two pieces.

In [4]:
Pi(1000)

Pi with 1000 steps is 3.141593 in 0.000338 secs


Let's now try a new tool: ```numba```! Numba can optimize specific portions of your code. Find out more here: http://numba.pydata.org/

In [5]:
from numba import jit 

@jit
def loop_numba(num_steps):
    step = 1.0/num_steps
    sum = 0    
    for i in range(num_steps):
        x= (i+0.5)*step
        sum = sum + 4.0/(1.0+x*x) 
    return sum

def Pi_numba(num_steps ):
    start = time.time()
    sum = loop_numba(num_steps)
    pi = sum/num_steps
    end = time.time()
    print("Pi with %d steps is %f in %f secs" %(num_steps, pi, end-start))
    
def Pi(num_steps):
    start = time.time()
    sum = loop(num_steps)
    pi = sum/num_steps
    end = time.time()
    print("Pi with %d steps is %f in %f secs" %(num_steps, pi, end-start))

In [6]:
print("without numba")
Pi(1000)
print("with numba")
Pi_numba(1000)

without numba
Pi with 1000 steps is 3.141593 in 0.000287 secs
with numba
Pi with 1000 steps is 3.141593 in 0.281114 secs


Try with different number of steps, e.g., 100000, 10000000, etc... How the two functions compare? Why?

# Go parallel?

If all optimization options, such as Numba, fail or do not provide the expected performance, you may try to parallelize your code. In other words, you should rewrite your code in such a way that specific portions of the code are split and executed simultaneously. Parallelization in python has some peculiarities, though.

First, the most common Python implementation, CPython, does not support multi-threading, i.e., does not allow multiple threads (roughly speaking, pieces of the executable code that share memory) to run concurrently. This is due to the fact that CPython has a Global Interpreter Lock, i.e. a lock that protects the code from thread-unsafe usages. The GIL makes sure that one thread can run at a time. However, many CPU-intensive python libraries, such as numpy and scipy, are multi-threading internally (not exposed to the interpreter). 

You can force your python code to be multi-threading using the ```threading``` library. However, because of the GIL, you won't get any advantage over a standard single-threading run.

In [7]:
# single_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 2.9584288597106934


To be compared with:

In [8]:
# multi_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 2.7884793281555176


Not really a great performance!

Exercise: try to make the code that computes Pi multi-threading. Tip: check the documentation of ```threading``` or browse the web for help!

In general, multi-threading is not the right option if you are CPU-bounded, i.e., if your bottleneck is given by computationally demanding tasks. It may be useful if you are I/O (input/output)-bounded, i.e., if your code has to wait a lot to get input or give output.

A possible way out in case of CPU-bounded codes is to use multi-processing instead of multi-threading. Roughly speaking, with multi-processing you will have multiple copies of your code running in parallel. This overcomes the GIL limitation, since each process will have its own GIL. However, it can be highly resources-demanding, because each process will have its own access to memory, copy of variables and objects that may be passed across processes etc. 

If you are on a single machine or CPU, you can use the ```multiprocessing``` library:

In [9]:
from multiprocessing import Pool
import time

COUNT = 50000000
def countdown(n):
    while n>0:
        n -= 1

if __name__ == '__main__':
    pool = Pool(processes=2)
    start = time.time()
    r1 = pool.apply_async(countdown, [COUNT//2])
    r2 = pool.apply_async(countdown, [COUNT//2])
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)

Time taken in seconds - 1.5433399677276611


This is about a factor 2 improvement with respect to the single-threading code. Note that it is not exactly twice as fast as the original code, since each process has its own overhead (preliminary things that has to be done before the core of the computation, hence the part that benefit more from the multi-processing, is run). This means that there might be a scaling issue as the number of processes increases, which may represent a bottleneck.

If you have the possibility to run on a cluster (of machines), hence to distribute your processes across multiple machines, you have to resort to other libraries, such ```MPI```. There is a specific implementation for python: ```mpi4py```.

If you have digested these information so far, you might want to take a deeper look to this lecture here: https://www.nesi.org.nz/sites/default/files/mpi-in-python.pdf
From Section 3 on, in particular, you will find a lot more information on parallel programming in python.

# Acknowledgements

This brief and down-to-earth intro is heavily based on a collection of materials from the web. In particular:

* intro to optimization tools (numba) and parallel programming: https://www.nesi.org.nz/sites/default/files/mpi-in-python.pdf
* Tutorial on GLI, multi-threading vs multi-processing: https://realpython.com/intro-to-python-threading/, https://realpython.com/python-gil/, https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know/

We would like to warmly thank Enrico Calore from INFN Ferrara for help in navigating the huge amount of available resources on the web and pointing us to the links above.